# Operational Research Assignment 2025
## Optimal Warehouse Positioning
### Lolos Ioannis - 10674
### lolosioann@ece.auth.gr

In [6]:
import numpy as np

warehouses = list(range(12))
customers = list(range(12))

# Fixed costs for warehouses (in thousands of euros)
fixed_cost = [3500, 9000, 10000, 4000, 3000, 9000, 9000, 3000, 4000, 10000, 9000, 3500]

# Capacities (in tons)
capacity = [300, 250, 100, 180, 275, 300, 200, 220, 270, 250, 230, 180]

# Demands of each customer (in tons)
demand = [120, 80, 75, 100, 110, 100, 90, 60, 30, 150, 95, 120]


In [7]:
transport_cost = np.array([
    [100,  80,  50,  50,  60, 100, 120,  90,  60,  70,  65, 110],
    [120,  90,  60,  70,  65, 110, 140, 110,  80,  80,  75, 130],
    [140, 110,  80,  80,  75, 130, 160, 125, 100, 100,  80, 150],
    [160, 125, 100, 100,  80, 150, 190, 150, 130,  np.inf,  np.inf,  np.inf],
    [190, 150, 130,  np.inf,  np.inf,  np.inf, 180, 150,  50,  50,  60, 100],
    [200, 180, 150,  np.inf,  np.inf,  np.inf, 100, 120,  90,  60,  75, 110],
    [120,  90,  60,  70,  65, 110, 140, 110,  80,  80,  75, 130],
    [120,  90,  60,  70,  65, 110, 140, 110,  80,  80,  75, 130],
    [140, 110,  80,  80,  75, 130, 160, 125, 100, 100,  80, 150],
    [160, 125, 100, 100,  80, 150, 190, 150, 130,  np.inf,  np.inf,  np.inf],
    [190, 150, 130,  np.inf,  np.inf,  np.inf, 200, 180, 150,  np.inf,  np.inf,  np.inf],
    [200, 180, 150,  np.inf,  np.inf,  np.inf, 100,  80,  50,  50,  60, 100]
])

# Convert transport cost to per-ton cost
cost_per_ton = np.zeros_like(transport_cost, dtype=float)

for i in warehouses:
    for j in customers:
        if transport_cost[i, j] == np.inf:
            cost_per_ton[i, j] = np.inf
        else:
            cost_per_ton[i, j] = transport_cost[i, j] / demand[j]



In [8]:
from gurobipy import Model, GRB

model = Model("warehouse_location")
model.setParam("OutputFlag", 0)  # silence output

y = model.addVars(warehouses, vtype=GRB.BINARY, name="open")
x = model.addVars(warehouses, customers, vtype=GRB.CONTINUOUS, name="flow")

model.setObjective(
    sum(fixed_cost[i] * y[i] for i in warehouses) +
    sum(cost_per_ton[i][j] * x[i, j]
        for i in warehouses for j in customers
        if cost_per_ton[i][j] < np.inf),
    GRB.MINIMIZE
)


Restricted license - for non-production use only - expires 2026-11-23


In [9]:
# Each customer must receive exactly their demand
for j in customers:
    model.addConstr(sum(x[i, j] for i in warehouses if cost_per_ton[i][j] < np.inf) == demand[j],
                    name=f"demand_{j}")

# Only open warehouses can ship, and can't exceed capacity
for i in warehouses:
    model.addConstr(sum(x[i, j] for j in customers if cost_per_ton[i][j] < np.inf) <= capacity[i] * y[i],
                    name=f"capacity_{i}")

# Disallow flows where cost is infinite
for i in warehouses:
    for j in customers:
        if cost_per_ton[i][j] == np.inf:
            model.addConstr(x[i, j] == 0)


In [10]:
model.optimize()

if model.status == GRB.OPTIMAL:
    print(f"\n✅ Total cost: {model.objVal:.2f} thousand euros\n")
    print("Opened Warehouses:")
    for i in warehouses:
        if y[i].X > 0.5:
            print(f"  Warehouse {i+1} (capacity: {capacity[i]})")

    print("\nCustomer Assignments:")
    for j in customers:
        print(f"  Customer {j+1} (demand: {demand[j]} tons):")
        for i in warehouses:
            flow = x[i, j].X
            if flow > 1e-6:
                print(f"    ← {flow:.1f} tons from Warehouse {i+1} (cost/ton: {cost_per_ton[i][j]:.2f})")
else:
    print("❌ No feasible solution found.")



✅ Total cost: 17929.23 thousand euros

Opened Warehouses:
  Warehouse 1 (capacity: 300)
  Warehouse 5 (capacity: 275)
  Warehouse 8 (capacity: 220)
  Warehouse 9 (capacity: 270)
  Warehouse 12 (capacity: 180)

Customer Assignments:
  Customer 1 (demand: 120 tons):
    ← 120.0 tons from Warehouse 1 (cost/ton: 0.83)
  Customer 2 (demand: 80 tons):
    ← 5.0 tons from Warehouse 1 (cost/ton: 1.00)
    ← 75.0 tons from Warehouse 8 (cost/ton: 1.12)
  Customer 3 (demand: 75 tons):
    ← 75.0 tons from Warehouse 1 (cost/ton: 0.67)
  Customer 4 (demand: 100 tons):
    ← 100.0 tons from Warehouse 1 (cost/ton: 0.50)
  Customer 5 (demand: 110 tons):
    ← 45.0 tons from Warehouse 8 (cost/ton: 0.59)
    ← 65.0 tons from Warehouse 9 (cost/ton: 0.68)
  Customer 6 (demand: 100 tons):
    ← 100.0 tons from Warehouse 8 (cost/ton: 1.10)
  Customer 7 (demand: 90 tons):
    ← 90.0 tons from Warehouse 12 (cost/ton: 1.11)
  Customer 8 (demand: 60 tons):
    ← 60.0 tons from Warehouse 12 (cost/ton: 1.33)
  C